<a href="https://colab.research.google.com/github/tamaramrt/Mamo.ia/blob/main/Treinamento_Classificadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SVM COM KERNEL RBF**

Instalação dos módulos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install pandas

In [ ]:
# importando as bibliotecas
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from torchvision import datasets
from torch.utils.data import DataLoader

Funções utilizadas para o processamento

In [ ]:
def preprocessamento_imagem(image_path):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # resize imagem para 28x28 (compatível com LeNet)
        transforms.ToTensor(),  # transforma em tensor
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    return image

def extrair_atributos_img(image_path, model):
    input_image = preprocessamento_imagem(image_path).to(device)
    model.eval()  # Colocar o modelo em modo de avaliação
    with torch.no_grad():
        # Passar pela conv1, conv2 e obter a penúltima camada
        x = model.pool(torch.relu(model.conv1(input_image)))
        x = model.pool(torch.relu(model.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        features = torch.relu(model.fc1(x))  # extrair os atributos da penúltima camada
    return features

Carregar o CSV dos atributos extraídos

In [ ]:

df = pd.read_csv('/content/drive/MyDrive/atributos_extracao_TERMOGRAFIA.csv') #colocar aqui o caminho

# Separar atributos (X) e rótulo (Y, a classe da amostra)

#remove a coluna class (ultima coluna) do dataframe, retornando somente os atributos
#converter esse df em um array numpy (.values)
#tornar X uma matriz com todas as colunas de atributos, que vai ser
#usada como entrada do modelo

X = df.drop(columns='class').values

#selecionar apenas a coluna class, que contem os rotulos (classes das imagens)
# depois converter em um array numpy

Y = df['class'].values




Dividir em treino e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

#test_size=0.2 (20% dos dados vão para teste, 80% para treino => confirmar isso)
#random_state=42 (garantir que a divisão será sempre igual toda vez que rodar)



Normalização dos dados

Treinar o classificador SVM com Kernel RBF

Fazer previsões

Avaliação do desempenho do modelo (métricas)

Matriz de confusão